In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
import torch.optim as optim
from torch.autograd import Variable
from time import time

from skimage.io import imread
from skimage.color import rgb2gray
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import torchvision.datasets as datasets
import skimage.io as io
from skimage import transform
from skimage.transform import resize, rescale, rotate, EuclideanTransform
import numpy as np

from scipy.signal import fftconvolve
from scipy import ndimage

from unet import *
from smokestack import *

In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on GPU.


In [3]:
# Predefine input size 
n_channels = 3
n_classes = 2
# Next creat an instance of the UNet model
modelUnet = UNet().to(device)
modelUnet.load_state_dict(torch.load("smokestack_segment.pt"))

<All keys matched successfully>

In [5]:
plt.figure(figsize=(12, 6))
size = (256, 256)
K = 50
for i in range(1, 7577, 5):
    if i < 600:
        K = 150
    else:
        K = 50
    I = io.imread("Drone/Frames/{}.png".format(i))
    X = np.array(np.moveaxis(resize(I, size), -1, 0), dtype=np.float32)
    dataloader = DataLoader(list(zip([X], [X])), batch_size=1, shuffle=True)
    for X, _ in dataloader:
        Y = F.sigmoid(modelUnet(X.to(device))).detach().cpu()
    Y = np.reshape(Y, (Y.shape[-2], Y.shape[-1]))
    Y = resize(Y, (I.shape[0], I.shape[1]))
    Y = np.array(Y*255, dtype=np.uint8)
    Y = Y > 200
    # Expand the mask
    Y2 = fftconvolve(Y, np.ones((K, K)), mode='same')/K**2
    Mask = Y2 > 1/K**2
    ILabel, NLabels = ndimage.label(Mask)
    idx = np.argmax(ndimage.sum(Mask, ILabel, range(NLabels+1)))
    Mask = (ILabel == idx)
    
    I[Mask == 0, :] = 255
    io.imsave("Drone/DroneFrames/{}.png".format(i), I)
    #I[Y > 200, 0] = Y[Y > 200]
    #newsize = (int(I.shape[0]/4), int(I.shape[1]/4))
    #I = resize(I, newsize)
    #io.imsave("Pred{}.png".format(i), I)

<Figure size 864x432 with 0 Axes>